In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [16]:
import os
import pandas as pd

# Load the training data
os.chdir('C:\\Users\\KIIT\\Downloads')
train_data = pd.read_csv('Train_Data.csv')
test_data = pd.read_csv('Test_Data.csv')
sample_submission = pd.read_csv('Sample_Submission.csv')

# Display the first few rows of the training data
train_data.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,...,0.06,0.06,0.00,0.00,0.00,0.0,1.00,1.0,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,...,0.45,0.04,0.00,0.00,0.11,0.0,0.02,0.0,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,...,1.00,0.00,0.02,0.03,0.00,0.0,0.00,0.0,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,...,0.01,0.08,0.00,0.00,0.00,0.0,1.00,1.0,19,1


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Define the target and features
X = train_data.drop(columns=['attack'])
y = train_data['attack']

# Define the categorical and numerical columns
categorical_cols = ['protocoltype', 'service', 'flag']
numerical_cols = X.columns.difference(categorical_cols)

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

# Predict on the validation set
y_pred = model_pipeline.predict(X_val)

# Evaluate the model
report = classification_report(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)

report, conf_matrix


('              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     10762\n           1       1.00      1.00      1.00      6607\n\n    accuracy                           1.00     17369\n   macro avg       1.00      1.00      1.00     17369\nweighted avg       1.00      1.00      1.00     17369\n',
 array([[10762,     0],
        [    0,  6607]], dtype=int64))

In [20]:
# Predict on the test data
test_data_transformed = model_pipeline.predict(test_data)

# Create a submission DataFrame
submission = pd.DataFrame({
    #'Id': sample_submission['Id'],
    'Attack': test_data_transformed
})

# Save the submission DataFrame to a CSV file
submission_path = 'Neptune_Attack_Prediction_Submission.csv'
submission.to_csv(submission_path, index=False)

submission.head(), submission_path


(   Attack
 0       1
 1       0
 2       1
 3       1
 4       1,
 'Neptune_Attack_Prediction_Submission.csv')

In [30]:
# Check if 'Attack' column exists in the training data
if 'Attack' in train_data.columns:
    print("Attack column found!")
else:
    print("Attack column not found. Please check the column names.")

# Check for missing values
print(train_data.isnull().sum())

# Drop or fill missing values
train_data = train_data.dropna()

from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
le = LabelEncoder()

# Encode categorical features in training data
for col in train_data.select_dtypes(include=['object']).columns:
    if col != 'Attack':  # Exclude the target column
        train_data[col] = le.fit_transform(train_data[col])

# Apply the same encoding to test data
for col in test_data.select_dtypes(include=['object']).columns:
    test_data[col] = le.fit_transform(test_data[col])

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Split the dataset into features and target variable
X = train_data.drop('Attack', axis=1)
y = train_data['Attack']

# Convert target variable to numerical
y = le.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Calculate the F1 score
f1 = f1_score(y_val, y_pred, pos_label=1)  # Assuming 'neptune' is encoded as 1
print('F1 Score:', f1)


Attack column not found. Please check the column names.
duration                  0
protocoltype              0
service                   0
flag                      0
srcbytes                  0
dstbytes                  0
land                      0
wrongfragment             0
urgent                    0
hot                       0
numfailedlogins           0
loggedin                  0
numcompromised            0
rootshell                 0
suattempted               0
numroot                   0
numfilecreations          0
numshells                 0
numaccessfiles            0
numoutboundcmds           0
ishostlogin               0
isguestlogin              0
count                     0
srvcount                  0
serrorrate                0
srvserrorrate             0
rerrorrate                0
srvrerrorrate             0
samesrvrate               0
diffsrvrate               0
srvdiffhostrate           0
dsthostcount              0
dsthostsrvcount           0
dsthostsamesrvrate  

KeyError: "['Attack'] not found in axis"